In [1]:
import pandas as pd
from underthesea import sent_tokenize
from underthesea import word_tokenize
import re
df = pd.read_json('news_dataset.json')
df = df['content']
df = df.sample(frac = 0.5, random_state=42, ignore_index=True)
df = df.drop_duplicates(ignore_index=True)

In [2]:
stopwords = stopwords = ["bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện", "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều", "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", "một_cách", "này", "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", "qua", "ra", "rằng", "rằng", "rất", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", "trước", "từ", "từng", "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"]

sent = set()
## Sentence tokenization
for i in range(len(df)):
    sentence = sent_tokenize(df.loc[i])
    for s in sentence:
        sent.add(s)
sent = list(sent)

In [3]:
for i in range(100000, len(sent)):
    sent[i] = re.sub('[\W]', ' ', sent[i].lower())
    tokens = []
    for t in word_tokenize(sent[i]):
        t = t.replace(" ", "_")
        if t not in stopwords:
            tokens.append(t)
    sent[i] = tokens

In [4]:
import json
file_path = 'array_data.json'

with open(file_path, mode='w', encoding='utf-8') as file:
    json.dump(sent, file, ensure_ascii=False, indent=4)

print(f"Dữ liệu đã được lưu tại {file_path}")

Dữ liệu đã được lưu tại array_data.json


In [5]:
import json
from gensim.models import Word2Vec
# Đọc dữ liệu từ file JSON
with open('array_data.json', mode='r', encoding='utf-8') as file:
    sent = json.load(file)
# Huấn luyện mô hình Word2Vec
pretrained_model = Word2Vec(
    sentences=sent,  # Dữ liệu huấn luyện
    vector_size=300,            # Số chiều vector nhúng
    window=3,                   # Cửa sổ ngữ cảnh
    min_count=3,                # Bỏ qua các từ xuất hiện < 3 lần
    sg=1,                       # Sử dụng Skip-gram
    hs=0,                       # Không dùng hierarchical softmax
    negative=10,                # Số mẫu âm tính
    epochs=10,                  # Số vòng lặp qua dữ liệu
    workers=3                  # Số luồng xử lý
)
# Lưu mô hình
pretrained_model.save("pretrained_word2vec.model")

In [7]:
pretrained_model.wv.most_similar(positive="nữ_sinh", topn = 6)

[('nam_sinh', 0.6152075529098511),
 ('thảo_ngân', 0.5380651354789734),
 ('học_sinh', 0.527435839176178),
 ('em', 0.5261192321777344),
 ('đăng_khải', 0.5126875042915344),
 ('nữ_sinh_học', 0.5105963349342346)]